## Project : CREDICT CARD FRAUD DETECTION

### Problem **Fraudelent Trasactions cost Bank millions of Dollars yearly**.

##### **Business Understanding (problem)** : 

💼 **Business Understanding Summary**: Credit Card Fraud Detection

The Problem

**Credit card fraud is when unauthorized or illegal transactions are made using someone else's card. For banks and financial institutions, this results in**:

* 💸 **Financial Losses**: Billions lost annually from chargebacks and reimbursements.

* 🛡️ **Customer Trust Issues**: If users feel unsafe, they may stop using the card or switch banks.

* 🔍 **Operational Costs**: Investigating fraud cases and tightening security costs time and money.

* ⚖️**Regulatory Pressure**: Banks must comply with laws around fraud protection and customer reimbursement.

**Business Goal**

Minimize financial losses and protect customers by detecting fraudulent transactions in real-time, without rejecting too many valid transactions (false positives).


## **Data Understanding**

## 📊 **Data Understanding** : Credit Card Fraud Detection Dataset

📦 **Dataset Source**

📍 **Origin**: The dataset is made available on [Kaggle](https://www.kaggle.com/datasets/rashmitachauhan/creditcardfrauddetection/data) from a research collaboration with European cardholders.


👥 Scope: Transactions made by European cardholders.

📁 Dataset Details

Filename: creditCardFraud_28011964_120214.csv


🔢 Features (Columns)

Column	

**Note** : Original dataset got it from `kaggle.com` but loaded the dataset in **Database** to practice sql skill

**Tools used** :

* SQL - queries dataset from database
* Python - data preparation, data cleaning etc
* Tableau - to communicate my inisights


In cell below import libraries to use




In [2]:
# import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

The cell below I step the connection to my **database** to load data there for working with

In [14]:
from mysql import connector
from dotenv import load_dotenv
import os

load_dotenv()
# PASSWORD = os.getenv("PASSWORD")

try:
    with connector.connect(
        host="localhost",
        user="root",
        password="",
    ) as database:
        # Create database if it doesn't exist
        create_db = "CREATE DATABASE IF NOT EXISTS `Bank_db`"
        with database.cursor() as cursor:
            cursor.execute(create_db)
            print("✅ Database created or already exists.")

        # Show all existing databases
        show_existing_db = "SHOW DATABASES"
        with database.cursor() as cursor:
            cursor.execute(show_existing_db)
            print("📦 Existing databases:")
            for db in cursor:
                print(f" - {db[0]}")

except connector.Error as err:
    print(f"❌ Error: {err}")


✅ Database created or already exists.
📦 Existing databases:
 - bank_db
 - information_schema
 - my_wordpress
 - mysql
 - performance_schema
 - phpmyadmin
 - portfolio
 - test
